Justin Ruths, jxr163730

In [1]:
import networkx as nx
import sys
sys.path.append('../d3networkx/')
import d3networkx as d3nx
from d3graph import D3Graph, D3DiGraph
from numpy import *
from time import time
import asyncio

def square_grid(n,d3,G,x0=100,y0=100,w=50):
    if G is None:
        G = D3Graph()
    # find the dimensions for the grid that are as close as possible
    num_rows = int(floor(sqrt(n)))
    while n % num_rows != 0:
        num_rows += 1
    num_cols = int(n/num_rows)
    
    # Add all the nodes
    G.add_nodes_from(range(n))
        
    # Add the edges and position the nodes
    for i in range(num_rows):
        for j in range(num_cols):
            n = num_cols*i + j
            d3.position_node(n,x0+i*w,y0+j*w)
            if i < num_rows-1:
                G.add_edge(n,n+num_cols) # add edge down
            if j < num_cols-1:
                G.add_edge(n,n+1) # add edge right

async def propagate(G,d3,x,steps,slp=0.5,keep_highlights=False,update_at_end=False):
    interactive = d3.interactive
    d3.set_interactive(False)
    A = nx.adjacency_matrix(G).todense().T  # adjacency matrix
    d3.highlight_nodes_by_index(list(where(x>0)[0]))
    d3.update()
    await asyncio.sleep(slp)
    cum_highlighted = sign(x)
    for i in range(steps): # the brains
        x = sign(dot(A,x)) # the brains
        cum_highlighted = sign(cum_highlighted+x)
        if not update_at_end:
            if not keep_highlights:
                d3.clear_highlights()
            d3.highlight_nodes_by_index(list(where(x>0)[0]))
            d3.update()
            await asyncio.sleep(slp)
    if update_at_end:
        if not keep_highlights:
            d3.clear_highlights()
            d3.highlight_nodes_by_index(list(where(x>0)[0]))
        else:
            d3.highlight_nodes_by_index(list(where(cum_highlighted>0)[0]))
        d3.update()
    d3.set_interactive(interactive)
    if keep_highlights:
        return cum_highlighted
    else:
        return x 

This next line starts up the visualizer. It will start some background code that sends data to the visualizer and then it will open a new browser window where the visualizer will live. Once you have the visualizer running, you can leave it running for the entire session, so don't re-run this block. If you close the `visualizer.html` (or hit refresh), you will need to reestablish this connection. In this case, you should click the refresh button in the Jupyter notebook (not for the webpage) to restart the kernel (which will clear your variables and Python environment).

In [2]:
d3 = await d3nx.create_d3nx_visualizer()
#d3 = await d3nx.create_d3nx_visualizer(canvas_size=(1200,1000))

websocket server started...networkx connected...visualizer connected...

#### Launching the Visualizer Manually

If the visualizer does not launch automatically, then you'll need to open it manually. After running the line above, use the following line to determine the communication port that the visualizer is using:

In [ ]:
d3.port

The port is a 4-digit number. Go to the file *visualizer.html* in the d3networkx folder. Double click on it to open it (do not open it in JupyterLabs). In the url, add the following (without the quotes) to the end of the url: "&port=1234" and replace 1234 with the 4-digit port above. A different port is selected each time you run the `d3 = await d3nx.create_d3nx_visualizer()` command. So as long as you don't run that command again (or restart the kernel, or close JupyterLab, that port should still continue to work)

## Grid Network

In [3]:
d3.clear()
d3.set_interactive(False)
G = D3Graph()
d3.set_graph(G)
square_grid(144,d3,G,x0=75,y0=70)
d3.update()

In [4]:
x = zeros((G.number_of_nodes(),1))
x[0] = 1
await propagate(G,d3,x,10,slp=1);

## Directed Network

In [8]:
d3.clear()
G = D3DiGraph(nx.read_weighted_edgelist('lab2.edgelist',create_using=nx.DiGraph))
d3.set_graph(G)
d3.update()

In [9]:
# code to propagate

In [10]:
# code to find the out-component of node 1

## E. coli Protein Network

In [5]:
d3.clear()
G = D3DiGraph(nx.read_weighted_edgelist('ecoli.edgelist',create_using=nx.DiGraph))
d3.set_interactive(False)
d3.set_graph(G)
d3.set_interactive(True)
d3.update()
print('Ecoli has %i nodes.' % G.number_of_nodes())

Ecoli has 418 nodes.


In [4]:
# code to find the out-component of node 2

In [5]:
# code to find the out-component of node 16

In [6]:
#print(nx.diameter(G)) # does not work!

def diameter2(G):
    spaths = dict(nx.all_pairs_shortest_path(G))
    # fill in!
    return diameter, path

# use the new diameter function here

## Section 6.12: Flows & Cut Sets

In [4]:
def worst_graph(n):
    # to do
    return G

def best_graph(n):
    # to do
    return G    

# create graph
start_time = time()
# perform min cut
print('min cut took %1.2f seconds' % (time() - start_time))

In [7]:
G = nx.read_gml('pert.gml','name')

# find longest path